# 8.对象引用，可变性与垃圾回收
## 8.1变量不是盒子

In [1]:
a = [1,2,3]
b = a
a.append(4)
b

[1, 2, 3, 4]

In [2]:
id(a) == id(b)

True

In [3]:
class Gizmo:
    
    def __init__(self):
        print('Gizmo id:{}'.format(id(self)))

x = Gizmo()


Gizmo id:1926989181256


In [4]:
y = Gizmo()

Gizmo id:1926989082440


## 8.2标识，相等性和别名

In [5]:
charles = {'nane':'Charles L', 'born':1832}
l = charles
l['age'] = 950

In [6]:
l is charles

True

In [7]:
charles

{'nane': 'Charles L', 'born': 1832, 'age': 950}

In [8]:
alex = {'nane':'Charles L', 'born':1832, 'age': 950}
alex == charles

True

In [9]:
alex is charles

False

### 8.2.2 元组的相对不可变性

In [15]:
t1 = (1,2,[3,4])
t2 = (1,2,[3,4])
t1 == t2

True

In [16]:
t1 is t2

False

In [18]:
id(t1)

1926989327480

In [19]:
id(t1[-1])

1926988474952

In [20]:
t1[-1].append(4)

In [21]:
id(t1)

1926989327480

In [22]:
id(t1[-1])

1926988474952

## 8.3 默认做浅复制

In [23]:
l1 = [1, [2, 3], (4, 5, 6)]
l2 = list(l1)# l2 = l1[:]

In [24]:
l2

[1, [2, 3], (4, 5, 6)]

In [25]:
l1 == l2, l1 is l2

(True, False)

In [26]:
l1 = [1, [2, 3], (4, 5, 6)]
l2 = list(l1)
l1.append(7)
l1[1].remove(2)
l1, l2

([1, [3], (4, 5, 6), 7], [1, [3], (4, 5, 6)])

In [27]:
l1[1] += [8, 9]
l2[2] += (10, 11)
l1, l2

([1, [3, 8, 9], (4, 5, 6), 7], [1, [3, 8, 9], (4, 5, 6, 10, 11)])

#### 为任意对象做深复制与浅复制

In [28]:
class Bus:
    
    def __init__(self, passengers = None):
        if passengers is None:
            self.passengers = []
        else:
            self.passengers = list(passengers)
    
    def pick(self, name):
        self.passengers.append(name)
        
    def remove(self, name):
        self.passengers.remove(name)

In [29]:
import copy
bus1 = Bus(['A', 'B', 'C', 'D'])
bus2 = copy.copy(bus1)
bus3 = copy.deepcopy(bus1)
id(bus1), id(bus2), id(bus3)

(1926989385352, 1926989385224, 1926989385288)

In [30]:
bus1.remove('B')
bus2.passengers

['A', 'C', 'D']

In [31]:
bus3.passengers

['A', 'B', 'C', 'D']

In [32]:
a = [1, 2]
b = [a, 3]
a.append(b)

In [33]:
a

[1, 2, [[...], 3]]

## 8.4 函数的参数作为引用时

In [34]:
def f(a, b):
    a += b
    return a
x = 1
y = 2
f(x, y)

3

In [35]:
x,y

(1, 2)

In [36]:
a = [1, 2]
b = [3, 4]
f(a, b)

[1, 2, 3, 4]

In [37]:
a,b

([1, 2, 3, 4], [3, 4])

In [38]:
s = (1, 2)
t = (3, 4)
f(s,t)

(1, 2, 3, 4)

In [39]:
s,t

((1, 2), (3, 4))

### 8.4.1 不要使用可变类型作为参数的默认值

In [48]:
class HBus:
    
    def __init__(self, passengers = []):
        self.passengers = passengers
    
    def pick(self, name):
        self.passengers.append(name)
        
    def remove(self, name):
        self.passengers.remove(name)

In [49]:
bus1 = HBus(['B', 'C'])
bus1.pick("A")
bus1.remove('B')

In [50]:
bus1.passengers

['C', 'A']

In [51]:
bus2 = HBus()
bus2.passengers

[]

In [52]:
bus2.pick('Z')
bus2.passengers

['Z']

In [53]:
bus3 = HBus()
bus3.passengers

['Z']

In [54]:
bus3.pick('D')
bus3.passengers

['Z', 'D']

In [55]:
bus2.passengers

['Z', 'D']

In [56]:
dir(HBus.__init__)

['__annotations__',
 '__call__',
 '__class__',
 '__closure__',
 '__code__',
 '__defaults__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__get__',
 '__getattribute__',
 '__globals__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__kwdefaults__',
 '__le__',
 '__lt__',
 '__module__',
 '__name__',
 '__ne__',
 '__new__',
 '__qualname__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__']

In [57]:
HBus.__init__.__defaults__

(['Z', 'D'],)

### 8.4.2 防御可变参数

In [60]:
class TBus:
    
    def __init__(self, passengers = None):
        if passengers is None:
            self.passengers = []
        else:
            self.passengers = passengers
        
    def pick(self, name):
        self.passengers.append(name)
    
    def drop(self, name):
        self.passengers.remove(name)


In [61]:
b_t = ['s', 't', 'm', 'd', 'p']
bus = TBus(b_t)
bus.drop('s')
bus.drop('t')
b_t

['m', 'd', 'p']

In [62]:
class TBus1:
    
    def __init__(self, passengers = None):
        if passengers is None:
            self.passengers = []
        else:
            self.passengers = list(passengers)
        
    def pick(self, name):
        self.passengers.append(name)
    
    def drop(self, name):
        self.passengers.remove(name)


In [64]:
b_t1 = ['s', 't', 'm', 'd', 'p']
bus1 = TBus1(b_t1)
bus1.drop('s')
bus1.drop('t')
b_t1

['s', 't', 'm', 'd', 'p']

In [65]:
bus.passengers is b_t

True

In [66]:
bus1.passengers is b_t1

False

## 8.5 垃圾回收

In [67]:
import weakref

s1 = {1, 2, 3}
s2 = s1

def bye():
    print('Gone with the wind')
    
ender = weakref.finalize(s1, bye)
ender.alive

True

In [68]:
del s1
ender.alive

True

In [69]:
s2 = 'spam'

Gone with the wind


In [70]:
ender.alive

False

## 8.6 弱引用

In [79]:
import weakref
a = {0, 1}
wref = weakref.ref(a)
wref

<weakref at 0x000001C0A98DC4A8; to 'set' at 0x000001C0A97E8D68>

In [80]:
wref()

{0, 1}

In [84]:
a = { 2, 3, 4}
wref()

{0, 1}

In [85]:
wref()  is None

False

In [86]:
wref() is None

False

In [87]:
wref() is None

False

In [88]:
_

False

In [89]:
a

{2, 3, 4}

In [90]:
wref is None

False

### 8.6.1 WeakValueDictionary简介

In [91]:
class Cheese:
    
    def __init__(self, kind):
        self.kind = kind
    
    def __repr__(self):
        return 'Cheese({})'.format(self.kind)
    
import weakref
stock = weakref.WeakValueDictionary()
catalog = [Cheese('R'), Cheese('t'), Cheese('b'), Cheese('P') ]
for cheese in catalog:
    stock[cheese.kind] = cheese
    
sorted(stock.keys())

['P', 'R', 'b', 't']

In [92]:
del catalog
sorted(stock.keys())

['P']

In [93]:
del cheese
sorted(stock.keys())

[]

### 8.6.2 弱引用的局限

In [95]:
class MyList(list):
    """"""
a = list(range(10))
m_a = MyList(range(10))
a, m_a

([0, 1, 2, 3, 4, 5, 6, 7, 8, 9], [0, 1, 2, 3, 4, 5, 6, 7, 8, 9])

In [97]:
wref1 = weakref.ref(m_a)
wref1()

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9]

In [98]:
wref2 = weakref.ref(a)
wref2()

TypeError: cannot create weak reference to 'list' object

In [99]:
d = dict()
wrefd = weakref.ref(d)
wrefd()

TypeError: cannot create weak reference to 'dict' object

## 8.7 Python对不可变对象的把戏

In [101]:
t1 = (1, 2, 3)
t2 = tuple(t1)
t3 = t1[:]
t1 is t3

True

In [102]:
t2 is t3

True

In [103]:
t1 = (1,2,3)
t2 = (1,2,3)
t1 is t2

False

In [104]:
s1 = '123'
s2 = '123'
s1 is s2

True